
# Advanced Data Science Capstone

## Correlation of air pollution and Prevalence of Heart failures in Germany  

## Data cleansing and Feature Creation

### The deliverables
The deliverables of the current stage:

 - current notebook as the process documentation
 - Pandas dataframe of the "wide" type, containing time series of pollutants concentrations and the unique sensor ID and a county id
 - Pandas dataframe with disease prevalence column(s) (heart failures,...) and a county id
 - Pandas dataframe with disease prevalence column(s), county id, and some features extracted from air pollution data series for sensors located in corresponding county

### Data cleansing
 - The air quality data sets are claimed to be "validated", so most work for cleansing the data is already done.
 - The incomplete files from the datasets (not having "hour" in the name) are ignored.
 - Few missing values appearing in the time series as negative values of the pollutant concentrations will be imputed.
 
###  Feature creation
The basic features for air pollution levels are

 - Number of hours when pollutant concentration exceeded some certain value
 - Mean or Median concentration of the pollutant
 
 First of all the necessary libraries and pre-defined functions are loaded:

In [118]:
import urllib.request
import xml.etree.ElementTree as ET
from lxml import etree
import pandas as pd
import numpy as np

import re, collections
from io import StringIO
import os, fnmatch, fastparquet

import matplotlib.pyplot as plt

def SelectAllXMLsensorID():
    varFull = [s for s in AllTags if 'value' in s][0]
    return([re.sub(r'[^a-zA-Z0-9:]*\'{http(.*)$', r'', re.sub(r'^.*AQD\/SPO.DE_', r'', str(varr.attrib))) for varr in Eroot.iter(varFull) if 'AQD' in str(varr.attrib)]) 



Now the files with pollutant concentration time series for the given year will be loaded to the **dffAll** Pandas dataframe of the **wide** format. During the load procedure **consistensy** of **files** and **column** names will be checked.

In [110]:
AirE1aDir='Capstone.rawData/AQD_DE_E1a_2017/'

#!ls Capstone.rawData/AQD_DE_E1a_2017/*hour*
FilesHour=[]

for file in os.listdir(AirE1aDir):
    if fnmatch.fnmatch(file, '*hour*'):
        FilesHour.append(file)
print("Number of files in the dataset", len(FilesHour))

# shortening the process for debugging purposes
#FilesHour=FilesHour[0:3]        

dffAll=pd.DataFrame(index=range(0,8760))  # 8760 hours in the year

# add observation periods:
file=FilesHour[0]
Etree = ET.parse(AirE1aDir+file)
Eroot = Etree.getroot()
Eroot.tag
Eroot.attrib
AllTags = [elem.tag for elem in Eroot.iter()]
varFull = [s for s in AllTags if 'values' in s][0]
varr=Eroot.iter(varFull)
dffAll=pd.concat([dffAll, dfftmp[0]], axis=1)
dffAll.columns=['observation_period']

# get all tags in xml file; Note, that the actual data is kept as a TEXT of *values* tags 
for file in FilesHour:
    Etree = ET.parse(AirE1aDir+file)
    Eroot = Etree.getroot()
    Eroot.tag
    Eroot.attrib
    AllTags = [elem.tag for elem in Eroot.iter()]
    
    ColNamesExp=SelectAllXMLsensorID()
# Compare column names with file names, they should encode same country, state and pollutant
    for ColName in ColNamesExp:
        if ((ColName[0:2]!=file[0:2]) or (ColName[2:4]!=file[3:5]) or (ColName[8:11]!=file[11:14])):
            print("Inconsistency in file and column names: ", file, ColName)
            exit()
    
    varFull = [s for s in AllTags if 'values' in s][0]
    dff=[]
    for varr in Eroot.iter(varFull):
        dff.append(pd.read_csv(StringIO((varr.text).replace("@@","\n")), sep=",", header=None))

# checking, that measurment timestamps are identical in the files read       
    for s in range(0,len(dff)):
        if not (dffAll['observation_period']).equals(dff[s][0]):
            print("Inconsistency of observation times in the following files: ", file, FilesHour[0])
            exit()

        
# select column 4 - pollutant concentration:
    dffTimeStr=dff[0][0]
    dff=pd.concat([dff[s][4] for s in range(0,len(dff))], axis=1)
    dff.columns=ColNamesExp
    
            
    
    dffAll=pd.concat([dffAll, dff], axis=1)

Number of files in the dataset 156


In [111]:
print("Memory usage: ", (dffAll.memory_usage(index=True).sum()/1048576.0), " MB")
dffAll.describe()

Memory usage:  151.5784454345703  MB


,DESH001_O3_dataGroup1,DESH008_O3_dataGroup1,DESH013_O3_dataGroup1,DESH014_O3_dataGroup1,DESH015_O3_dataGroup1,DESH016_O3_dataGroup1,DESH023_O3_dataGroup1,DESH033_O3_dataGroup1,DESH035_O3_dataGroup1,DESH056_O3_dataGroup1,...,DEHH015_PM1_dataGroup1,DEHH016_PM1_dataGroup1,DEHH026_PM1_dataGroup1,DEHH033_PM1_dataGroup1,DEHH059_PM1_dataGroup1,DEHH068_PM1_dataGroup1,DEHH070_PM1_dataGroup1,DEHH072_PM1_dataGroup1,DEHH079_PM1_dataGroup1,DEHH081_PM1_dataGroup1
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,...,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,45.374908,48.709722,37.286848,60.931687,42.575316,-397.154575,39.948094,-511.639592,37.867500,-28.696564,...,-28.273549,-8.032516,-55.883309,-12.692748,-28.061636,16.247620,3.343157,-28.703924,-0.194743,-10.264745
std,50.596504,73.491147,149.589802,33.518547,85.258260,511.246865,104.336547,525.860031,70.716164,281.257366,...,211.948821,155.066454,269.333205,170.167351,208.851818,87.331396,125.651302,209.301279,139.332240,174.797689
min,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,...,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000
25%,29.517250,37.534750,44.862250,46.691750,32.350750,-999.000000,32.580250,-999.000000,26.056250,29.833500,...,8.667750,7.454750,10.366000,8.229000,7.656000,11.960750,10.458750,6.427250,11.132500,10.722500
50%,48.199500,53.988500,59.384500,65.548000,49.974000,8.216000,50.504500,-999.000000,42.929000,52.865000,...,13.801000,12.723500,16.290500,13.302000,12.888500,19.360000,15.781500,11.748500,15.707000,16.113500
75%,64.562000,68.677500,70.980000,77.226500,64.915250,41.180250,66.478000,55.730000,56.653250,68.088000,...,21.444750,20.192750,25.062250,20.403500,20.664000,30.738000,23.237250,20.787250,22.993000,23.949250
max,145.660000,131.607000,138.455000,154.401000,143.936000,142.818000,144.126000,127.519000,124.948000,132.191000,...,275.973000,190.385000,266.052000,152.603000,196.997000,469.227000,277.583000,113.427000,170.415000,537.024000


Now we have **wide** dataframe, containing timeseries of all pollutant concentrations for all sensors. The pollutant type and the sensor ID are encoded in column names. The minimal value of pollutant concentrations *-999.0* is equivalent to *NA* and will be imputted:

In [112]:
dffAll[dffAll == -999.0] = np.NaN
dffAll.interpolate(method='linear', inplace=True, axis=0)
dffAll['observation_period']=pd.to_datetime(dffAll['observation_period'])
dffAll['observation_period']=dffAll['observation_period'].dt.to_period('H')
dffAll.describe()
#dffAll['observation_period'][0].end_time

,DESH001_O3_dataGroup1,DESH008_O3_dataGroup1,DESH013_O3_dataGroup1,DESH014_O3_dataGroup1,DESH015_O3_dataGroup1,DESH016_O3_dataGroup1,DESH023_O3_dataGroup1,DESH033_O3_dataGroup1,DESH035_O3_dataGroup1,DESH056_O3_dataGroup1,...,DEHH015_PM1_dataGroup1,DEHH016_PM1_dataGroup1,DEHH026_PM1_dataGroup1,DEHH033_PM1_dataGroup1,DEHH059_PM1_dataGroup1,DEHH068_PM1_dataGroup1,DEHH070_PM1_dataGroup1,DEHH072_PM1_dataGroup1,DEHH079_PM1_dataGroup1,DEHH081_PM1_dataGroup1
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8329.000000,...,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,47.300995,53.445478,58.428197,61.540782,49.019535,37.805624,49.843107,34.955394,42.253745,52.193195,...,17.683511,16.011442,20.536382,16.477444,16.459830,23.509776,18.934294,16.133223,19.077072,20.227036
std,23.766766,22.207443,18.970011,21.952156,23.288963,17.652888,23.687623,25.095800,21.012580,22.863523,...,13.782866,12.728282,13.465247,11.916048,12.867936,16.734014,13.445291,13.370300,11.937898,17.229521
min,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,...,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000
25%,29.737500,37.898000,45.848750,46.744750,32.776750,31.523898,33.350000,17.042000,26.418000,36.909000,...,9.351571,7.818000,11.777000,8.694250,8.260500,12.088500,10.776750,6.992000,11.439000,11.257524
50%,48.250500,54.144500,59.600000,65.574500,50.124500,38.926364,50.706500,17.042000,43.008000,54.761000,...,14.220500,12.970000,17.072500,13.665500,13.178500,19.528000,16.040500,12.080000,15.916800,16.298000
75%,64.576000,68.786750,71.177500,77.229750,64.967500,45.751475,66.523000,55.801750,56.704750,68.704000,...,21.774000,20.420000,25.698250,20.523515,20.837000,31.056750,23.545250,21.251500,23.239000,24.205750
max,145.660000,131.607000,138.455000,154.401000,143.936000,142.818000,144.126000,127.519000,124.948000,132.191000,...,275.973000,190.385000,266.052000,152.603000,196.997000,469.227000,277.583000,113.427000,170.415000,537.024000


Now we can save the resulting dataset for further use:


In [124]:
#dffAll.dtypes
#fastparquet.write('Capstone.ETL/Capstone.etl.wide.1.0.parquet', dffAll)
dffAll.to_csv('Capstone.ETL/Capstone.etl.wideCSV.1.0.gzip', compression='gzip')
#pd.Period(pd.to_datetime("2017-01-01T00:00:00+01:00"), freq='H').end_time
#pd.to_datetime(dffTimeStr[0:2])

### Sensor Locations
In order to use the spatial data one should have coordinates of air pollution measurements sensors.
For the current study the county index for every individual sensor is needed. First all measurement stations IDs and the town names of the sensors locations are read to **SensorLocation** dataframe:

In [207]:
# pick all tags from the XML file
Etree = etree.parse("Capstone.rawData/AQD_DE_D_2017/DE_D_allInOne_metaMeasurements_2017.xml")
Eroot = Etree.getroot()
Eroot.tag
Eroot.attrib
AllTags = [elem.tag for elem in Eroot.iter()]

# get correct tag names for 'municipality', 'EUStationCode' and 'featureMember':
varMUN = [s for s in AllTags if 'municipality' in s][0]
varID  = [s for s in AllTags if 'EUStationCode' in s][0]
varFeatMem = [s for s in AllTags if 'featureMember' in s][0]

IDs=[]
MUNs=[]
# read 'municipality' and 'EUStationCode' to SensorLocation dataframe:
for varr in Eroot.iter(varFeatMem):
    for child in varr.iter(varMUN):
        MUNs.append(child.text)
        for child2 in varr.iter(varID):
            IDs.append(child2.text)
SensorLocation=pd.DataFrame({'SensorID': IDs, 'SensorTown': MUNs})
SensorLocation.tail(5)

,SensorID,SensorTown
803,DEUB005,Lüder
804,DEUB028,Zingst
805,DEUB029,Suhl
806,DEUB030,Stechlin
807,DEUB044,Garmisch-Partenkirchen


In order to map town names to county names, used in the health related datasets, the town-county table **dfCT** will be created. It contains 5-digit county-id (not unique, but characterizing counties in some vicinity), name of town and county: 

In [208]:
columns = [(10, 15), (22, 71), (72, 121)]
dfCT = pd.read_fwf("Capstone.rawData/GV100AD3107/GV100AD_310719.ASC", 
                     colspecs=columns, names=['countyid','town','county'],
                     encoding="iso8859_1")
dfCT=dfCT.fillna(method='ffill')

dfCT['town'] = dfCT['town'].str.split(",").str[0]
dfCT.tail(5)

,countyid,town,county
16116,16077,Starkenberg,Schmölln/Thür.
16117,16077,Thonhausen,Schmölln/Thür.
16118,16077,Treben,Schmölln/Thür.
16119,16077,Vollmershain,Schmölln/Thür.
16120,16077,Windischleuba,Schmölln/Thür.


### Prevalence of Heart failures
The central dataframe of the model will contain list of counties, prevalence of disease(s) in this counties, and the set of air-pollution-based features. Let's load the *Prevalence of Heart failures* dataset: 

In [209]:
xlsx_file = pd.ExcelFile("Capstone.rawData/Heart_2017/data_id_97_kreis11_2_j_1483228800.xlsx")
print("xls sheet names: ",xlsx_file.sheet_names)
dfHeart = xlsx_file.parse('Daten', header=3, decimal=",") 
print(dfHeart.head(3))
print("Number of duplicates in Regions-ID column: ", dfHeart.duplicated(['Regions-ID']).sum())

xls sheet names:  ['Hintergrundinformationen', 'Daten']
             Region  Regions-ID  KV           Kreistyp  Wert  Bundeswert
0            Lk.Hof        9475  BY  Ländliches Umland  6.43        3.11
1  Mansfeld-Südharz       15087  ST    Ländlicher Raum  6.37        3.11
2               Hof        9464  BY  Ländliches Umland  6.36        3.11
Number of duplicates in Regions-ID column:  0


The mapping will start from setting the **countyID** to every **sensorID** in the **SensorLocation** dataframe:


In [210]:
SensorLocation = (SensorLocation.join(dfCT[['countyid','town']].set_index('town'),
                                      on='SensorTown')).drop_duplicates(subset=['SensorID'])

Checking the resulting table it was found, that 30 of 804 entries have not resolved **countyid**:

In [211]:
print("Total number of sensors: ", SensorLocation.count())
print("Number of sensors with unresolved countyid: ", SensorLocation[SensorLocation.isna().any(axis=1)].count())
#print("List of unresolved sensors:")
#SensorLocation[SensorLocation.isna().any(axis=1)]
#print("Number of duplicates in SensorID column: ", SensorLocation.duplicated(['SensorID']).sum())
#SensorLocation.loc[SensorLocation.duplicated(['SensorID'])==True]

Total number of sensors:  SensorID      808
SensorTown    808
countyid      778
dtype: int64
Number of sensors with unresolved countyid:  SensorID      30
SensorTown    30
countyid       0
dtype: int64


At the moment it is easier to drop these 4% of sensor's data. Otherwise this table could be corrected manually, since it has reasonable size, and it's contents (sensor lables/county codes) hardly changes in time. 

In [216]:
SensorLocation=SensorLocation.dropna()
SensorLocation=SensorLocation.astype({'countyid':int})
SensorLocation.head(5)

,SensorID,SensorTown,countyid
0,DEBB007,Elsterwerda,12062
1,DEBB021,Potsdam,12054
2,DEBB026,Spremberg,12071
3,DEBB029,Schwedt/Oder,12073
4,DEBB032,Eisenhüttenstadt,12067


In [217]:
SensorLocation.count()

SensorID      778
SensorTown    778
countyid      778
dtype: int64

In [218]:
dffAll.head(3)

,observation_period,DESH001_O3_dataGroup1,DESH008_O3_dataGroup1,DESH013_O3_dataGroup1,DESH014_O3_dataGroup1,DESH015_O3_dataGroup1,DESH016_O3_dataGroup1,DESH023_O3_dataGroup1,DESH033_O3_dataGroup1,DESH035_O3_dataGroup1,...,DEHH015_PM1_dataGroup1,DEHH016_PM1_dataGroup1,DEHH026_PM1_dataGroup1,DEHH033_PM1_dataGroup1,DEHH059_PM1_dataGroup1,DEHH068_PM1_dataGroup1,DEHH070_PM1_dataGroup1,DEHH072_PM1_dataGroup1,DEHH079_PM1_dataGroup1,DEHH081_PM1_dataGroup1
0,2017-01-01 00:00,7.601,16.634,20.710,18.211,10.385,3.218,9.248,20.068,12.890,...,249.448,44.554,266.052,75.456,192.427,469.227,277.583,52.103,71.111,46.581
1,2017-01-01 01:00,8.931,16.785,25.177,22.396,12.437,2.234,8.496,22.947,12.708,...,275.973,80.144,247.447,152.603,196.997,141.295,225.350,47.672,170.415,71.369
2,2017-01-01 02:00,10.546,16.538,26.634,27.768,13.529,9.859,5.924,20.721,11.885,...,59.263,51.266,51.252,58.362,51.243,98.176,48.333,51.111,97.089,49.493
